### Initialization

In [1]:
import dotenv
import os

from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

### AI Agent

In [2]:
manual_input_questions = [
    "We received a request from a distributor to create a piggyback for part numbers XYZ123. Can you confirm if any additional uplift is required?",
    "Customer is unable to retrieve the quote document from GPMS. Can you verify the quote and trigger a TEIS ticket if needed?",
    "There is an issue with TE.com where the customer cannot raise SPRs for PN XYZ456. Can you advise on the next steps?"
]

yes_no_questions = [
    "Can we replace POS for quote #500xxxxx as per the request?",
    "The opportunity #500xxxx has been rejected on SFDC. Should the customer raise a new opportunity?",
    "Quote #500xxxxx has been closed by BUPA. Should we escalate this to them?"
]

predefined_response_questions = [
    "What is the predefined response for a general pricing query related to a quote extension?",
    "How do we handle an enquiry regarding a rejected S&D claim?",
    "What is the process for adding a new part number to an existing piggyback agreement?"
]

wiki_search_questions = [
    "What is an OEM agreement and how does it relate to piggyback creation?",
    "Can you explain the process of ship & debit in the context of pricing adjustments?",
    "What is GPMS, and how does it work for quote management?"
]

fallback_questions = [
    "A distributor is requesting additional pricing information for an expired quote. How should we proceed?",
    "Customer is facing issues with updating LOA on their account. Can you assist?",
    "A quote has been marked as closed in GPMS but is still pending in SFDC. What action should be taken?"
]

if __name__ == "__main__":
    for question in yes_no_questions:
        inputs = {
            "question": "Can we replace POS for quote #500xxxxx as per the request?"
        }
        
        for output in app.stream(inputs):
            for key, value in output.items():
                if "response" in value:
                    print("\nGenerated Email:\n")
                    print(value["response"])
                else:
                    print(f"Node '{key}': {value}")



NameError: name 'app' is not defined

In [ ]:
import mysql.connector
import pandas as pd

# MySQL connection details (Update these)
MYSQL_HOST = "localhost"
MYSQL_USER = "root"
MYSQL_PASSWORD = "12345678"  # Update with your actual MySQL password
MYSQL_DATABASE = "te_project_test_1"

# 1️⃣ Read the CSV file
csv_file_path = "ship_debit_queries.csv"  # Change this to your actual CSV path

# Try using ',' as delimiter first. If data still looks wrong, switch to '\t'.
df = pd.read_csv(csv_file_path, delimiter=",", encoding="utf-8")

# Print actual column names to check parsing
print("Actual Column Names:", df.columns.tolist())

# 🔹 Strip spaces, replace special characters, and convert to lowercase
df.columns = (
    df.columns.str.strip()
    .str.replace(" ", "_")  # Replace spaces with underscores
    .str.replace("&", "and")  # Replace '&' with 'and'
    .str.lower()
)

# Print after cleaning
print("Cleaned Column Names:", df.columns.tolist())

# 🔹 Correct Column Mapping  
column_mapping = {
    "quote_id": "Quote ID",
    "fsa_to_sandd_conversion": "FSA to S&D Conversion",  # Fixed '&' issue
    "pos_customer": "POS Customer",
    "end_customer": "End Customer",
    "address_issue": "Address Issue",
    "quote_closed_by": "Quote Closed By",
    "next_action_required": "Next Action Required",
    "additional_findings": "Additional Findings"
}

# Rename columns to match MySQL table schema
df.rename(columns={v: k for k, v in column_mapping.items()}, inplace=True)

# Ensure only relevant columns are selected
df = df[list(column_mapping.keys())]

# ✅ Convert DataFrame to list of tuples explicitly
data_tuples = [tuple(x) for x in df.to_numpy()]

# Debugging: Print a sample of the tuples
print("Sample Data:", data_tuples[:5])  # Print first 5 rows to verify format

# 2️⃣ Connect to MySQL and insert data
try:
    connection = mysql.connector.connect(
        host=MYSQL_HOST, user=MYSQL_USER, password=MYSQL_PASSWORD, database=MYSQL_DATABASE
    )
    cursor = connection.cursor()

    # Define SQL INSERT query
    insert_query = """
    INSERT INTO quotes (Quote_ID, FSA_to_S_D_Conversion, POS_Customer, End_Customer, 
                        Address_Issue, Quote_Closed_By, Next_Action_Required, Additional_Findings)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """

    # Insert data into MySQL
    cursor.executemany(insert_query, data_tuples)
    connection.commit()

    print("✅ CSV data uploaded successfully!")

except mysql.connector.Error as err:
    print(f"❌ Error: {err}")

finally:
    cursor.close()
    connection.close()


# Dump

In [3]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langgraph.graph import END, StateGraph, START
import mysql.connector
from datetime import datetime
import os
import re

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY is not set")

llm = ChatGroq(groq_api_key=GROQ_API_KEY, model_name="llama-3.3-70b-versatile")

class RouteQuery(BaseModel):
    """Route a user query to the correct processing logic."""
    category: Literal[
        "pos_replace",
        "general_pricing_queries",
        "piggyback_creation",
        "adding_parts_to_piggyback",
        "ship_and_debit_queries",
        "opportunities_rejected_sfdc",
        "pending_approval_sfdc",
        "quote_closed_gpms_no_document",
        "quote_not_reaching_pricing",
        "customer_data_enquiries",
        "quotes_pending_review_gpms",
        "opportunities_pending_review_sfdc",
        "opportunity_rejected_incorrectly_sfdc",
        "loa_related_queries",
        "s_and_d_claim_rejection",
        "agreement_pn_addition_removal",
        "te_com_issues",
        "product_enquiry",
        "feedback",
        "complaint",
        "fallback"
    ] = Field(..., description="Classify the user query into one of 21 specific routes.")

def get_ship_debit_info(quote_id):
    """Fetch Ship & Debit related information from the database with enhanced error handling and logging."""
    try:
        print(f"🔌 Connecting to database for quote ID: {quote_id}")
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            password="12345678",
            database="te_project_test_1"
        )
        cursor = connection.cursor(dictionary=True)
        
        # First, get table structure to understand available columns
        cursor.execute("DESCRIBE quotes")
        columns = cursor.fetchall()
        column_names = [col['Field'] for col in columns]
        print(f"📋 Available columns in 'quotes' table: {', '.join(column_names)}")
        
        # Query the quote
        query = """SELECT * FROM quotes WHERE Quote_ID = %s"""
        cursor.execute(query, (quote_id,))
        result = cursor.fetchone()
        
        if result:
            print(f"✅ Quote found! Quote ID: {quote_id}")
            # Print all columns and values for debugging
            for key, value in result.items():
                print(f"  • {key}: {value}")
            return result
        else:
            print(f"❌ No quote found with ID: {quote_id}")
            
            # Try to see if the quote exists with a different case or format
            cursor.execute("SELECT * FROM quotes LIMIT 5")
            sample_data = cursor.fetchall()
            
            if sample_data:
                print("📊 Sample data from quotes table:")
                for row in sample_data:
                    if 'Quote_ID' in row:
                        print(f"  Sample Quote_ID: {row['Quote_ID']}")
            
            return None
            
    except mysql.connector.Error as err:
        print(f"❌ Database Error: {err}")
        return None
    finally:
        if 'connection' in locals() and connection.is_connected():
            cursor.close()
            connection.close()
            print("🔌 Database connection closed")

def process_ship_debit_query(query):
    """Process Ship & Debit queries and generate an appropriate response with better error handling."""
    print(f"🔍 Processing query: {query}")
    
    # Extract quote ID - handle both formats with or without # prefix
    quote_match = re.search(r'#?(\d{10})', query)
    if not quote_match:
        return "⚠️ Could not find a valid Quote ID in the query. Please provide a **10-digit** Quote ID."
    
    quote_id = quote_match.group(1)
    print(f"📝 Extracted Quote ID: {quote_id}")
    
    quote_info = get_ship_debit_info(quote_id)
    
    if not quote_info:
        return f"""
📅 **Date:** {datetime.now().strftime("%B %d, %Y")}

 **No detailed information found for Quote ID: {quote_id}**

Possible reasons:
- The quote may not exist in our database
- There might be a database connection issue
- The quote ID format might be incorrect

Please verify the quote ID and try again, or contact technical support if the issue persists.

**Best Regards,**  
TE Connectivity Support Team
"""
    
    field_mappings = {
        "fsa_to_sandd_conversion": ["fsa_to_sandd_conversion", "fsa_to_s&d_conversion", "fsa_conversion"],
        "pos_customer": ["pos_customer", "pos_cust", "customer_pos"],
        "end_customer": ["end_customer", "end_cust", "customer_end"],
        "address_issue": ["address_issue", "address_problem", "address_status"],
        "quote_closed_by": ["quote_closed_by", "closed_by", "processor"],
        "next_action_required": ["next_action_required", "next_action", "action_required"],
        "additional_findings": ["additional_findings", "findings", "notes", "comments"]
    }
    
    extracted_data = {}
    for field, possible_columns in field_mappings.items():
        value = "N/A"
        for col in possible_columns:
            if col in quote_info and quote_info[col] is not None and quote_info[col] != "":
                value = quote_info[col]
                print(f"✓ Found data for '{field}' in column '{col}': {value}")
                break
        extracted_data[field] = value
    
    is_bupa_closed = False
    closed_by = extracted_data["quote_closed_by"].lower()
    if closed_by != "n/a" and ("bupa" in closed_by or "business partner" in closed_by):
        is_bupa_closed = True
        print("🔍 Identified as BUPA-closed quote")
    
    if is_bupa_closed:
        response = f"""
📅 **Date:** {datetime.now().strftime("%B %d, %Y")}

🔹 **Quote ID:** {quote_id}  
🔹 **FSA to S&D Conversion:** {extracted_data['fsa_to_sandd_conversion']}  
🔹 **POS Customer:** {extracted_data['pos_customer']}  
🔹 **End Customer:** {extracted_data['end_customer']}  
🔹 **Address Issue:** {extracted_data['address_issue']}  

🛑 **This quote has been closed by BUPA.**  

**Next Steps:**  
 {extracted_data['next_action_required']}  
 **Additional Findings:** {extracted_data['additional_findings']}  

🔗 Please direct further queries to **BUPA** for more information.  

**Best Regards,**  
TE Connectivity Support Team
"""
    else:
        response = f"""
📅 **Date:** {datetime.now().strftime("%B %d, %Y")}

🔹 **Quote ID:** {quote_id}  
🔹 **FSA to S&D Conversion:** {extracted_data['fsa_to_sandd_conversion']}  
🔹 **POS Customer:** {extracted_data['pos_customer']}  
🔹 **End Customer:** {extracted_data['end_customer']}  
🔹 **Address Issue:** {extracted_data['address_issue']}  
🔹 **Quote Closed By:** {extracted_data['quote_closed_by']}  

**Next Steps:**  
{extracted_data['next_action_required']}  
**Additional Findings:** {extracted_data['additional_findings']}  

🔗 Please let us know if you need any further assistance.  

**Best Regards,**  
TE Connectivity Support Team
"""
    return response

def handle_ship_debit_queries(state):
    """Handles ship and debit related queries."""
    query = state["question"]
    response = process_ship_debit_query(query)
    return {"response": response}

def handle_fallback(state):
    """Handles queries that don't match known categories."""
    return {"response": "⚠️ I'm sorry, but I couldn't classify your request. Can you rephrase it or provide more details?"}

workflow = StateGraph(dict)

workflow.add_node("ship_and_debit_queries", handle_ship_debit_queries)
workflow.add_node("fallback", handle_fallback)  # Define the fallback node

def route_question(state):
    """Determine which node to send the query to."""
    question = state["question"].lower()
    print(f"🧭 Routing question: {question}")
    
    if any(keyword in question for keyword in ["ship", "debit", "s&d", "fsa", "sandd"]) or re.search(r'\b\d{10}\b', question):
        print("Routed to: ship_and_debit_queries")
        return "ship_and_debit_queries"
    
    print("Routed to: fallback")
    return "fallback"

workflow.add_conditional_edges(
    START,
    route_question,
    {
        "ship_and_debit_queries": "ship_and_debit_queries",
        "fallback": "fallback"
    }
)


workflow.add_edge("ship_and_debit_queries", END)
workflow.add_edge("fallback", END)

app = workflow.compile()

if __name__ == "__main__":
    test_queries = [
        "Can you check the status of quote 5001414860 for ship and debit conversion?",
        "What's the status of FSA to S&D conversion for quote 5001414860?",
        "Need details about quote 5001414860 address issues"
    ]

    for test_query in test_queries:
        inputs = {"question": test_query}
        print(f"\n🔎 **Testing Query:** {test_query}")
        print("-" * 80)
        for output in app.stream(inputs):
            for key, value in output.items():
                if isinstance(value, dict) and "response" in value:
                    print(value["response"])
                else:
                    print(f" Node '{key}': {value}")
        print("-" * 80)

d:\Projects New\Tests\TE AI Cup\Automated-Email-Reply-System\venv\Lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)



🔎 **Testing Query:** Can you check the status of quote 5001414860 for ship and debit conversion?
--------------------------------------------------------------------------------
🧭 Routing question: can you check the status of quote 5001414860 for ship and debit conversion?
Routed to: ship_and_debit_queries
🔍 Processing query: Can you check the status of quote 5001414860 for ship and debit conversion?
📝 Extracted Quote ID: 5001414860
🔌 Connecting to database for quote ID: 5001414860
📋 Available columns in 'quotes' table: Quote_ID, FSA_to_S_D_Conversion, POS_Customer, End_Customer, Address_Issue, Quote_Closed_By, Next_Action_Required, Additional_Findings
✅ Quote found! Quote ID: 5001414860
  • Quote_ID: 5001414860
  • FSA_to_S_D_Conversion: Yes
  • POS_Customer: Company E Corp
  • End_Customer: End J Ltd
  • Address_Issue: No
  • Quote_Closed_By: BUPA
  • Next_Action_Required: Address it to BUPA
  • Additional_Findings: Approved for change
🔌 Database connection closed

📅 **Date:** March